# Analyze Medical Costs Using Regression Tools


### Author: Cera Drake
### Data: 04/20/2025
### Objective: We have a data set of medical costs for individuals. Using different tools, I am going to see if there is any correlation between certain personal factors and higher medical costs